## Kamada-Kawai layout in igraph, graphviz and networkx illustrated by Plotly plots of an unrooted tree ##

### Tree as an instance of the igraph.Graph class ###

In [1]:
import plotly.graph_objects as go
import plotly.io as pio
import igraph as ig
pio.templates.default='none'
from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot
init_notebook_mode(connected=True)

In [2]:
G = ig.Graph.Read_GML('Data/zika.gml')
V = range(len(G.vs))

In [3]:
E = [e.tuple for e in G.es]# list of edges
labels = [v['name'] for v in G.vs]
layt = G.layout('kk') #kamada-kawai

Define data for the Plotly plot:

In [4]:
def get_plotly_data(E, coords):
    # E is the list of tuples representing the graph edges
    # coords is the list of node coordinates 
    N = len(coords)
    Xnodes = [coords[k][0] for k in range(N)]# x-coordinates of nodes
    Ynodes = [coords[k][1] for k in range(N)]# y-coordnates of nodes

    Xedges = []
    Yedges = []
    for e in E:
        Xedges.extend([coords[e[0]][0], coords[e[1]][0], None])
        Yedges.extend([coords[e[0]][1], coords[e[1]][1], None])
        
    return Xnodes, Ynodes, Xedges, Yedges 

def get_node_trace(x, y, labels, marker_size=5, marker_color='#6959CD', 
                   line_color='rgb(50,50,50)', line_width=0.5):
    return go.Scatter(
                x=x,
                y=y,
                mode='markers',
                marker=dict(
                            size=marker_size, 
                            color=marker_color,
                            line=dict(color=line_color, width=line_width)
                             ),
            text=labels,
            hoverinfo='text'
               )

def get_edge_trace(x, y, linecolor='rgb(210,210,210)', linewidth=1):
    return go.Scatter(
                x=x,
                y=y,
                mode='lines',
                line=dict(color=linecolor, width=linewidth),
                hoverinfo='none'
               )
    

In [5]:
Xn, Yn, Xe, Ye = get_plotly_data(E, layt)
trace1 = get_edge_trace(Xe, Ye)
trace2 = get_node_trace(Xn, Yn, labels)

Set the plot layout:

In [6]:
title1= "Unrooted tree"
width=800
height=800
layout=go.Layout(title=title1,
            font_size=12,
            showlegend=False,
            width=width,
            height=height,
            xaxis_visible=False,
            yaxis_visible=False,          
            hovermode='closest',
            paper_bgcolor='#eeeeee',     
            annotations=[dict(showarrow=False, 
                               text='igraph Kamada-Kawai layout',  
                               xref='paper',     
                               yref='paper',     
                               x=0,  
                               y=-0.1,  
                               xanchor='left',   
                               yanchor='bottom',  
                               font=dict(size=14)     
                              )
                        ],           
            )


In [7]:
fig1 = go.Figure(data=[trace1, trace2], layout=layout)
iplot(fig1)

Display the number of weak connected components in this graph:

In [8]:
len(G.clusters( mode='weak'))

1

### Tree as an instance of the  pygraphviz.AGraph class ###

In [9]:
import pygraphviz as pgv

In [10]:
g = pgv.AGraph(strict=True, directed=False)
g.add_nodes_from(V)
g.add_edges_from(E)
g.layout(prog='neato') 

In [11]:
def get_node_coordinates(g):
    #convert the string containing the node coordinates to a list of two floats
    #returns the list of lists of node coordinates
    if  not isinstance(g, pgv.AGraph):
        raise ValueError('The graph g must be a pygraphviz AGraph')
    N = len(g.nodes())    
    coords=[]
    for k in range(N):
        s = g.get_node(k).attr['pos']
        t = s.split(",")
        coords.append([float(t[0]), float(t[1])])
    return coords  

In [12]:
pgvlayt = get_node_coordinates(g)

In [13]:
Xnode, Ynode, Xedge, Yedge = get_plotly_data(E, pgvlayt)
trace3 = get_edge_trace(Xedge, Yedge)
trace4 = get_node_trace(Xnode, Ynode, labels)

In [14]:
fig2 = go.Figure(data=[trace3, trace4], layout=layout)
#fig2.update_layout(annotations[0]['text']='Graphviz neato layout')
fig2['layout']['annotations'][0]['text'] = "Graphviz neato layout"
iplot(fig2)

### Tree as an instance of the networkx.Graph class###

In [15]:
import networkx as nx

In [16]:
H = nx.Graph()
H.add_nodes_from(V)
H.add_edges_from(E)


Get the node positions:

In [17]:
pos = nx.kamada_kawai_layout(H) 
nodes=pos.keys()

Define data for the  Plotly plot:

In [18]:
Xv = [pos[node][0] for node in nodes]
Yv = [pos[node][1] for node in nodes]
Xed = []
Yed = []
for edge in H.edges():
    Xed.extend([pos[edge[0]][0],pos[edge[1]][0], None])
    Yed.extend([pos[edge[0]][1],pos[edge[1]][1], None]) 

In [19]:
trace5 = get_edge_trace(Xed, Yed)
trace6 = get_node_trace(Xv, Yv, labels)

In [20]:
fig3 = go.Figure(data=[trace5, trace6], layout=layout)
fig3['layout'].update(width=800, height=800)
fig3['layout']['annotations'][0]['text'] = "Networkx Kamada-Kawai layout"
iplot(fig3)

Inspect the number of connected components computed via networkx:

In [21]:
print (nx.number_connected_components(H))

1


In this case the graph is connected and the KK-layout implemented in the three Python packages generates similar nodes placement, up to a transformation.

Comparing with the results of the experiment performed in this [Jupyter Notebook](http://nbviewer.jupyter.org/github/empet/Networks/blob/master/kk-layout-in-3-packages-netsci.ipynb) we are led to the conclusion that the `nx.kamada_kawada_layout` doesn't work with disconnected graphs. 

In [22]:
from IPython.core.display import HTML
def  css_styling():
    styles = open("./custom.css", "r").read()
    return HTML(styles)
css_styling()